# Demo 2 - Agentic AI

Let's try a different approach for using agentic AI

#### Import Libraries

In [ ]:
import os
from langchain_openai import AzureChatOpenAI

#### Set Certificate

In [ ]:
os.environ['SSL_CERT_FILE'] = 'ca-bundle-full.crt'

#### Create instance of our model and test if it works

In [ ]:
model = AzureChatOpenAI(
    openai_api_version="<API_VERSION>",
    deployment_name="gpt-4o-2024-08-06",
    azure_endpoint="<ENDPOINT_URL>",
    openai_api_type="azure",
    openai_api_key="<API_KEY>",

    temperature=1,
    max_tokens=500,
    top_p=0.5,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None
    )



In [ ]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

#### Set our tool

In [ ]:
from langchain_core.tools import tool

@tool #Try commenting this out to see if it works without the decorator
def count_how_many_letters(text: str, letter:str) -> int:
    """Counts how many letters of a specific kind are in the text."""
    return text.count(letter)

#count_how_many_letters("strawberry", "r")

In [ ]:
tools = [count_how_many_letters]

### Binding our tools to our model

For debugging

Let's test if our model and the tools can be called/invoked

In [ ]:
model_with_tools = model.bind_tools(tools)

response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")


In [ ]:
response = model_with_tools.invoke([HumanMessage(content="How many times does the letter 'r' appear in the word strawberry??")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

#### Create the React Agent

Note that we pass only the model and not the model_with_tools as create_react_agent binds it for us

In [ ]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

In [ ]:
response = agent_executor.invoke({"messages": [HumanMessage(content="How many times does the letter 'r' appear in the word strawberry??")]})

response["messages"]

#### Let's format the output better with streaming

In [ ]:
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="How many times does the letter 'r' appear in the word strawberry??")]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()



In [ ]:
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="what react agent tools can you use?")]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()



### Let's create more tools and let the AI Combine them

In [ ]:
from langchain_core.tools import tool

BASE_PATH = "Data/"

def is_path_safe(filename):
    return os.path.abspath(os.path.join(BASE_PATH, filename)).startswith(os.path.abspath(BASE_PATH))

@tool
def read_file(filename):
    """
    Reads and returns the contents of the given text file.
    If the file does not exist, returns an error message.
    Arguments:
        filename: The name of the file to read.
    Returns:
        The contents of the file as a string, or an error message if the file cannot be read.
    """
    if not is_path_safe(filename):
        return "Error: Unsafe file path."
    try:
        with open(filename, 'r', encoding='utf-8') as f:
            return f.read()
    except Exception:
        return f"Error reading {filename}"

@tool
def replace_in_text(old_char, new_char, text):
    """
    Replaces all occurrences of old_char with new_char in the specified file.
    Returns a message indicating success or failure.
    Arguments:
        old_char: The character to be replaced.
        new_char: The character to replace with.
        text: The text in which to perform the replacement.
    Returns:
        The modified text with replacements made.
    """
    text = text.replace(old_char, new_char)
    return text

@tool
def save_to_file(filename, content):
    """
    Saves the given string content to the specified file.
    Overwrites the file if it already exists.
    Returns a message indicating success or failure.
    Arguments:
        filename: The name of the file to save to.
        content: The string content to write to the file.
    Returns:
        A success message if the file was saved, or an error message if there was a problem.
    """
    if not is_path_safe(filename):
        return "Error: Unsafe file path."
    try:
        with open(filename, 'w', encoding='utf-8') as f:
            f.write(content)
        return f"Content successfully saved to {filename}."
    except Exception:
        return f"Error saving to {filename}"


In [ ]:
tools = [read_file, replace_in_text, save_to_file]

In [ ]:
agent_executor = create_react_agent(model, tools)

In [ ]:

for step in agent_executor.stream(
    {"messages": [HumanMessage(content="can you read Data/input.txt, get the people 17 and below, change the delimiter to commas, and save to Data/output.txt")]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

